In [43]:
!pip install jupyter-dash dash pandas plotly

In [44]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import dash
from dash import dcc, html, Input, Output

# Տվյալների բեռնում
mouse_data = pd.read_csv(r"C:\Users\Admin\Downloads\Mouse_metadata.csv")
study_results = pd.read_csv(r"C:\Users\Admin\Downloads\Study_results.csv")
merged_df = pd.merge(mouse_data, study_results, on="Mouse ID")

# Գույներ՝ ըստ դեղերի
drug_colors = {
    'Placebo': '#29304E',
    'Capomulin': '#27706B',
    'Ramicane': '#71AB7F',
    'Ceftamin': '#9F4440',
    'Infubinol': '#FFD37B',
    'Ketapril': '#FEADB9',
    'Naftisol': '#B3AB9E',
    'Propriva': '#ED5CD4',
    'Stelasyn': '#97C1DF',
    'Zoniferol': '#8980D4'
}

# App initialization
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H3("Chart 1: Weight Distribution by Drug Regimen"),
    
    dcc.Checklist(
        id='drug-selector',
        options=[{'label': drug, 'value': drug} for drug in sorted(mouse_data['Drug Regimen'].unique())],
        value=['Placebo'],
        labelStyle={'display': 'inline-block'}
    ),
    
    dcc.Graph(id='histogram')
])

# Callback
@app.callback(
    Output('histogram', 'figure'),
    Input('drug-selector', 'value')
)
def update_histogram(selected_drugs):
    traces = []
    for drug in selected_drugs:
        weights = mouse_data[mouse_data['Drug Regimen'] == drug]['Weight (g)']
        traces.append(go.Histogram(
            x=weights,
            name=drug,
            opacity=0.7,
            marker=dict(color=drug_colors.get(drug, '#888'))
        ))
        
    layout = go.Layout(
        barmode='stack',
        xaxis=dict(title='Mouse Weight (g)'),
        yaxis=dict(title='Number of Mice'),
        legend=dict(x=0, y=1)
    )
    
    return go.Figure(data=traces, layout=layout)

# Run locally
if __name__ == '__main__':
    app.run(debug=True)

In [45]:
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H3("Chart 2: Selected Drug vs All - Weight Distribution"),
    
    dcc.Dropdown(
        id='drug-dropdown',
        options=[{'label': drug, 'value': drug} for drug in sorted(mouse_data['Drug Regimen'].unique())],
        value='Placebo',
        clearable=False
    ),
    
    dcc.Graph(id='chart-2')
])

# Callback
@app.callback(
    Output('chart-2', 'figure'),
    Input('drug-dropdown', 'value')
)
def update_chart_2(selected_drug):
    fig = go.Figure()

    # Ամբողջ քաշի բաշխում
    fig.add_trace(go.Histogram(
        x=mouse_data['Weight (g)'],
        name='All Drugs',
        opacity=0.5,
        marker_color='gray'
    ))

    # Ընտրված դեղի բաշխում
    selected_weights = mouse_data[mouse_data['Drug Regimen'] == selected_drug]['Weight (g)']
    fig.add_trace(go.Histogram(
        x=selected_weights,
        name=selected_drug,
        opacity=0.7,
        marker_color='crimson'
    ))

    fig.update_layout(
        barmode='overlay',
        xaxis_title='Mouse Weight (g)',
        yaxis_title='Number of Mice',
        legend_title='Drug',
        title='Weight Distribution of Selected Drug vs All'
    )

    return fig

# Վազեցնելու համար՝ VS Code → app.run(debug=True), Jupyter → run_server(...)
if __name__ == '__main__':
    app.run(debug=True)

In [46]:
drug_groups = {
    'lightweight': ['Capomulin', 'Ramicane'],
    'heavyweight': ['Ceftamin', 'Infubinol'],
    'placebo': ['Placebo']
}

# App initialization
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H3("Chart 3: Survival Pattern by Weight per Drug Group"),
    
    dcc.Checklist(
        id='group-selector',
        options=[{'label': g.capitalize(), 'value': g} for g in drug_groups.keys()],
        value=['placebo'],
        labelStyle={'display': 'inline-block'}
    ),
    
    dcc.Graph(id='chart-3')
])

# Callback
@app.callback(
    Output('chart-3', 'figure'),
    Input('group-selector', 'value')
)
def update_chart_3(selected_groups):
    fig = go.Figure()

    for group in selected_groups:
        for drug in drug_groups[group]:
            filtered = merged_df[merged_df['Drug Regimen'] == drug]
            weights = filtered['Weight (g)']

            fig.add_trace(go.Histogram(
                x=weights,
                name=drug,
                opacity=0.6
            ))

    fig.update_layout(
        barmode='overlay',
        xaxis_title='Mouse Weight (g)',
        yaxis_title='Number of Mice',
        title='Weight-based Survival Pattern by Drug Group',
        legend_title='Drugs'
    )

    return fig

# Վազեցնել
if __name__ == '__main__':
    app.run(debug=True)

In [47]:
drug_groups = {
    'lightweight': ['Capomulin', 'Ramicane'],
    'heavyweight': ['Ceftamin', 'Infubinol'],
    'placebo': ['Placebo']
}

# App initialization
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H3("Chart 4: Survival Over Time by Drug Group"),

    dcc.Checklist(
        id='survival-group-selector',
        options=[{'label': k.capitalize(), 'value': k} for k in drug_groups.keys()],
        value=[],  # Case 1: Nothing is selected
        labelStyle={'display': 'inline-block'}
    ),

    dcc.Graph(id='chart-4')
])

# Callback for Chart 4
@app.callback(
    Output('chart-4', 'figure'),
    Input('survival-group-selector', 'value')
)
def update_chart_4(selected_groups):
    fig = go.Figure()

    for group in selected_groups:
        for drug in drug_groups[group]:
            # ֆիլտրում ենք ըստ դեղի
            subset = merged_df[merged_df['Drug Regimen'] == drug]

            # հաշվում ենք յուրաքանչյուր պահի ողջ մկների քանակը (այսինքն՝ յուրահատուկ մկներ timepoint-ով)
            survival = subset.groupby("Timepoint")["Mouse ID"].nunique().reset_index(name="Mice Alive")

            fig.add_trace(go.Scatter(
                x=survival["Timepoint"],
                y=survival["Mice Alive"],
                mode="lines+markers",
                name=drug
            ))

    fig.update_layout(
        title="Number of Alive Mice Over Time",
        xaxis_title="Time Point (days)",
        yaxis_title="Number of Alive Mice",
        legend_title="Drugs",
        xaxis=dict(tickmode='linear', dtick=5)
    )

    return fig

# Վազեցնելու համար
if __name__ == '__main__':
    app.run(debug=True)